In [1]:
import io
import tempfile
import os
import pandas as pd
import re

In [2]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

In [3]:
import gspread
import subprocess

import PyPDF2
import requests

In [ ]:
df = pd.read_excel('../Data/Final_Thawra.xlsx', header=1, index_col='recordnumber')
df.head(2)

In [24]:
# df['drive'] = ''
# for _, row in df.iterrows():
#     for i in row.index:
#         if str(row[i]).startswith('https://drive.google'):
#             df.at[_,'drive'] = row[i]

In [25]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

In [26]:
def extract_text_from_pdf(pdf_path):
    # fh = io.BytesIO()
    # downloader = MediaIoBaseDownload(fh, response)
    # pdf_content = fh.getvalue()
    with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            for page_num in range(reader.numPages):
                page = reader.getPage(page_num)
                text += page.extractText()
    return text


In [4]:
def extract_file_id(url):
    pattern = r"open\?id=([\w-]+)"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        pattern = r"file/d/([\w-]+)"
        match = re.search(pattern, url)
        if match:
            return match.group(1)
        else:
            return None


In [5]:
def clean_text(pdf_text):
    
    #cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', pdf_text)
    
    #cleaned_text = re.sub(r'[^؀-ۿ\s]', '', pdf_text)
    cleaned_text = re.sub(r'http\S+', '', pdf_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'%[0-9a-fA-F]{2}', '', cleaned_text)
    cleaned_text = cleaned_text.replace('\n',' ').replace('ـ', '').replace('�', '')
    return cleaned_text

In [12]:
# Set up Google Sheets API credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheet = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json', scope)
client = gspread.authorize(creds_sheet)

# Fetch PDF URLs from Google Sheets document
spreadsheet_key = '1xyQe3vgYKGl_hcPa1MOEn970xjTUirEUq_ewfiGjLRs'
sheet_name = 'Main'
worksheet = client.open_by_key(spreadsheet_key).worksheet(sheet_name)


In [13]:
creds_drive = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service_drive = build('drive', 'v3', credentials=creds_drive)

In [14]:
urls = worksheet.col_values(161)
urls = [x for x in urls if len(x) > 6] 
len(urls)

10

In [15]:
urls = [x for x in urls if x]
len(urls)

10

In [16]:
urls

['https://drive.google.com/open?id=18oW7JuXmtxhOL6GJZH27CB6KS1isMbu5&authuser=amrdukmak%40gmail.com&usp=drive_fs',
 'https://drive.google.com/file/d/1DnP3136ezQuns9E9At4swskZHdpSCZgG/view?usp=share_link',
 'https://drive.google.com/file/d/1fLBkFDRUtgoEaGIx_gEXWuZdTTJ1cKBk/view?usp=share_link',
 'https://drive.google.com/file/d/1LoeXUXlFzI5KDJxhHZWc8MzZgJD2U7ng/view?usp=share_link',
 'https://drive.google.com/file/d/1WysX1w13fDSzSKa-t0Cd7x0HC6n67NJB/view?usp=sharing',
 'https://drive.google.com/file/d/1VIVZRvsKVRDsizc6IwAZMsur4e4H8Qi6/view?usp=sharing',
 'https://drive.google.com/file/d/1vZgAlr77hiPuSvXCVHH-21m_zselLcLF/view?usp=sharing',
 'https://drive.google.com/file/d/1kSgelIxklsHEFh3rn7YnoyqpT7NEFJtn/view?usp=sharing',
 'https://drive.google.com/open?id=1wiX2A9_Xc3mUOQOxAU7Oag_ep2xeThI4&authuser=amrdukmak%40gmail.com&usp=drive_fs',
 'https://drive.google.com/file/d/1FZboNDY8sUeP-Ec7S9dHhfPnR3EB4yu9/view?usp=sharing']

In [17]:
urls = list(dict.fromkeys(urls))
len(urls)

10

In [18]:
count = 0
for url in urls:
    file_id = extract_file_id(url)
    if file_id:
        try:
            request = service_drive.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                #print("Download %d%%." % int(status.progress() * 100))
            pdf_content = fh.getvalue()
            with open('new.pdf', 'wb') as f:
                f.write(pdf_content)

            pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
            
            pdf_text = ""
            for page in range(pdf._get_num_pages()):
                text = pdf.pages[page].extract_text()
                pdf_text = pdf_text + ''.join(text)
                
            cleaned_text = clean_text(pdf_text)
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 163, pdf_text)
            os.remove('new.pdf')
            count += 1
            print((count * 100)/len(urls) , url)
            
        except:
            print('error')
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 163, 'error')
    else:
        cell = worksheet.find(url)
        worksheet.update_cell(cell.row, 163, 'NOT_FOUND')

10.0 https://drive.google.com/open?id=18oW7JuXmtxhOL6GJZH27CB6KS1isMbu5&authuser=amrdukmak%40gmail.com&usp=drive_fs
20.0 https://drive.google.com/file/d/1DnP3136ezQuns9E9At4swskZHdpSCZgG/view?usp=share_link
30.0 https://drive.google.com/file/d/1fLBkFDRUtgoEaGIx_gEXWuZdTTJ1cKBk/view?usp=share_link
40.0 https://drive.google.com/file/d/1LoeXUXlFzI5KDJxhHZWc8MzZgJD2U7ng/view?usp=share_link
50.0 https://drive.google.com/file/d/1WysX1w13fDSzSKa-t0Cd7x0HC6n67NJB/view?usp=sharing
60.0 https://drive.google.com/file/d/1VIVZRvsKVRDsizc6IwAZMsur4e4H8Qi6/view?usp=sharing
70.0 https://drive.google.com/file/d/1vZgAlr77hiPuSvXCVHH-21m_zselLcLF/view?usp=sharing
80.0 https://drive.google.com/file/d/1kSgelIxklsHEFh3rn7YnoyqpT7NEFJtn/view?usp=sharing
90.0 https://drive.google.com/open?id=1wiX2A9_Xc3mUOQOxAU7Oag_ep2xeThI4&authuser=amrdukmak%40gmail.com&usp=drive_fs
100.0 https://drive.google.com/file/d/1FZboNDY8sUeP-Ec7S9dHhfPnR3EB4yu9/view?usp=sharing


In [ ]:
for url in bad_urls:
    file_id = extract_file_id(url)
    print(file_id)
    if file_id:
        try:
            request = service_drive.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                #print("Download %d%%." % int(status.progress() * 100))
            pdf_content = fh.getvalue()
            with open('new.pdf', 'wb') as f:
                f.write(pdf_content)

            pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
            pdf_text = ""

            pdf_text = pdf.pages[0].extract_text()
            text = clean_text(pdf_text)
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 167, text)
            os.remove('new.pdf')
            count += 1
            print((count * 100)/len(urls) , url)
        except:
            print('error')
            bad_urls.append(url)
            worksheet.update_cell(cell.row, 167, 'error')
    else:
        cell = worksheet.find(url)
        worksheet.update_cell(cell.row, 167, 'NOT_FOUND')

In [ ]:
url = 'https://drive.google.com/open?id=12wiwto6X1NpzNYBDjqnxdsuw9YjE-7Cb&authuser=amrdukmak%40gmail.com&usp=drive_fs'

file_id = extract_file_id(url)
print(url)
print(file_id)


In [ ]:
for url in urls:
    pdf_text = subprocess.check_output(['python', 'pdf_text_extractor.py', url])
    # Update Google Sheets with the extracted text
    cell = worksheet.find(url)
    worksheet.update_cell(cell.row, cell.col -2, pdf_text)

In [ ]:
# file_id = extract_file_id(urls[444])
# results = service_drive.files().get(fileId=file_id).execute()

# request = service_drive.files().export_media(fileId=results.get('id'), mimeType=results.get('mimeType'))

#request = service_drive.files().get_media(fileId=file_id)
# fh = io.BytesIO()
# downloader = MediaIoBaseDownload(fh, request)
# #print(request.headers)
# with open('new.pdf', 'wb') as f:
#     f.write(fh.getvalue())

# pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# pdf_text = ""

# for page in range(pdf.getNumPages()):
#     pdf_text += pdf.getPage(page).extractText()
# print(pdf_text)

In [ ]:
# response = requests.get(urls[300])
# print(response.headers)

#fh = io.BytesIO(response.content)
#downloader = MediaIoBaseDownload(fh, response.content)
    
# pdf_content = fh.read()
# pdf_content
# with open('new.pdf', 'wb') as f:
#     f.write(pdf_content)
    

# # pdf_content = response.content
# print(pdf_content)
# # with open('new.pdf', 'wb') as f:
# #     f.write(pdf_content)


# pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# pdf_text = ""
# for page in range(pdf.getNumPages()):
#     pdf_text += pdf.getPage(page).extractText()

# # # Print the extracted text
# print(pdf_text)

# # fh = io.BytesIO()
# # downloader = MediaIoBaseDownload(fh, pdf_content)
# # pdf_content = fh.getvalue()
# # with open('new.pdf', 'wb') as f:
# #         f.write(pdf_content)

# # pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# # pdf_text = extract_text_from_pdf('new.pdf')
# # pdf_text
# # # temp_file_path

In [ ]:
# Extract text from each PDF using the local Python function
